# Fine Tuning

We are going to try and fine tune a FCN classifier so to segment cells from histopathology scans. Our implementation is based on the implementation based on FCN.  


The fine tuning wil be addressed in four steps. Preparing the data in the form of blobs, so as to be caffee compatible but also in three model training. We will train with a 32 stride, take the parameters and plug them into a finer 16 stride model, and finally the same protocole is used to go a 8 stride finner model.

We will use the pre-train models available in the caffee zoo. We will remove each final layer and fine tune it via back propagation. 




# Preparing the data

Under the library caffee, data is provided to model under the form of blobs. A blob is a 4 dimensinal array where each channels represents: (num, channel, height, width). We will be providing cell segmentation data from histopathology slides. 
It can accept several types: HDF5, LMDB, etc..
Lets create an LMDB file to feed the neural network:

In [1]:
from DataToLMDB import MakeLMDB
import ImageTransf as Transf

path = '/home/naylor/Bureau/ToAnnotate'

output_dir = path + '/lmdb'

enlarge = False ## create symetry if the image becomes black ? 

transform_list = [Transf.Identity(),
                  Transf.Rotation(45, enlarge=enlarge), 
                  Transf.Rotation(90, enlarge=enlarge),
                  Transf.Rotation(135, enlarge=enlarge),
                  Transf.Flip(0),
                  Transf.Flip(1),
                  Transf.OutOfFocus(5),
                  Transf.OutOfFocus(10),
                  Transf.ElasticDeformation(0, 30, num_points = 4),
                  Transf.ElasticDeformation(0, 30, num_points = 4)]

MakeLMDB(path, output_dir, transform_list, val_num = 2, verbose = False)

1 / 40
2 / 40
3 / 40
4 / 40
5 / 40

/home/naylor/Python_pkg/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/home/naylor/Python_pkg/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/home/naylor/Python_pkg/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/home/naylor/anaconda/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))



6 / 40
7 / 40
8 / 40
9 / 40
10 / 40
11 / 40
12 / 40
13 / 40
14 / 40
15 / 40
16 / 40
17 / 40
18 / 40
19 / 40
20 / 40
21 / 40
22 / 40
23 / 40
24 / 40
25 / 40
26 / 40
27 / 40
28 / 40
29 / 40
30 / 40
31 / 40
32 / 40
33 / 40
34 / 40
35 / 40
36 / 40
37 / 40
38 / 40
39 / 40
40 / 40


# Preparing the 32 stride model: Pascal voc example

The architecture and pre-trained weight parameters can be found on the FCN github page. The architecture is based on the famous AlexNet implementation, the authors performed "net surgery" to transform the fully connected layers (final layers of the AlexNet model) into fully convolutionnal layers. These layers have the particularity of accepting any given size of inputs. Once these layers "convoloutionnized" , they added an upsampling and a deconvolution layer as to improve the accuracy of the model, indeed, AlexNet models provided the "what" but does not initially provide the where. Bridges were also added in order to combine low lever features to higher level features.